# PACKAGES AND LIBRARIES

In [ ]:
#GENERAL
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
#PATH PROCESS
import os
import os.path
from pathlib import Path
import glob
#IMAGE PROCESS
from PIL import Image
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.applications.vgg16 import preprocess_input, decode_predictions
import imageio
from IPython.display import Image
import matplotlib.image as mpimg
#MUSIC PROCESS
import pydub
from scipy.io.wavfile import read, write
import librosa
import librosa.display
import IPython
from IPython.display import Audio
import scipy
#SCALER & TRANSFORMATION
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import regularizers
from sklearn.preprocessing import LabelEncoder
#ACCURACY CONTROL
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
#OPTIMIZER
from keras.optimizers import RMSprop,Adam,Optimizer,Optimizer, SGD
#MODEL LAYERS
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D,BatchNormalization,\
                        Permute, TimeDistributed, Bidirectional,GRU, SimpleRNN,\
LSTM, GlobalAveragePooling2D, SeparableConv2D, ZeroPadding2D, Convolution2D, ZeroPadding2D,Reshape,\
Conv2DTranspose, LeakyReLU, Conv1D, AveragePooling1D, MaxPooling1D
from keras import models
from keras import layers
import tensorflow as tf
from keras.applications import VGG16,VGG19,inception_v3
from keras import backend as K
from keras.utils import plot_model
from keras.datasets import mnist
import keras
#SKLEARN CLASSIFIER
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
#IGNORING WARNINGS
from warnings import filterwarnings
filterwarnings("ignore",category=DeprecationWarning)
filterwarnings("ignore", category=FutureWarning) 
filterwarnings("ignore", category=UserWarning)

# PATH, LABEL, TRANSFORMATION

In [ ]:
Meta_Data_CSV = pd.read_csv("../input/urbansound8k/UrbanSound8K.csv")

In [ ]:
Path_Wav_List = []
Category_List = []

for path_number in range(8732):
    File_Path_Name = "../input/urbansound8k/fold" + str(Meta_Data_CSV["fold"][path_number]) + "/" + Meta_Data_CSV["slice_file_name"][path_number]
    Path_Wav_List.append(File_Path_Name)
    Category_List.append(Meta_Data_CSV["class"][path_number])

In [ ]:
Path_Wav_Series = pd.Series(Path_Wav_List,name="WAV").astype(str)
Category_Series = pd.Series(Category_List,name="CATEGORY")

In [ ]:
Main_Train_Data = pd.concat([Path_Wav_Series,Category_Series],axis=1)

In [ ]:
Main_Train_Data = Main_Train_Data.sample(frac=1).reset_index(drop=True)

# PROCESS FUNCTIONS

In [ ]:
def noise_function(data):
    noise_value = 0.009 * np.random.uniform() * np.amax(data)
    data = data + noise_value * np.random.normal(size=data.shape[0])
    
    return data



def stretch_function(data,rate=0.8):
    
    return librosa.effects.time_stretch(data,rate)



def shift_function(data):
    shift_range = int(np.random.uniform(-5,5) * 1000)
    
    return np.roll(data,shift_range)



def pitch_function(data,sampling_rate,pitch_factor=0.5):
    
    return librosa.effects.pitch_shift(data,sampling_rate,pitch_factor)



def extract_function(data):
    
    output_result = np.array([])
    mean_zero = np.mean(librosa.feature.zero_crossing_rate(y=data).T,axis=0)
    output_result = np.hstack((output_result,mean_zero))
    
    stft_output = np.abs(librosa.stft(data))
    chroma_output = np.mean(librosa.feature.chroma_stft(S=stft_output,sr=sample_rate).T,axis=0)
    output_result = np.hstack((output_result,chroma_output))
    
    mfcc_output = np.mean(librosa.feature.mfcc(y=data,sr=sample_rate).T,axis=0)
    output_result = np.hstack((output_result,mfcc_output))
    
    root_output = np.mean(librosa.feature.rms(y=data).T,axis=0)
    output_result = np.hstack((output_result,root_output))
    
    mel_output = np.mean(librosa.feature.melspectrogram(y=data,sr=sample_rate).T,axis=0)
    output_result = np.hstack((output_result,mel_output))
    
    return output_result



def export_function(path):
    
    data,sample_rate = librosa.load(path,duration=1.0)
    
    output_One = extract_function(data)
    result = np.array(output_One)
    
    noise_output = noise_function(data)
    output_Two = extract_function(noise_output)
    result = np.vstack((result,output_Two))
    
    stretch_output = stretch_function(data)
    stretch_pitch = pitch_function(stretch_output,sample_rate)
    output_Three = extract_function(stretch_pitch)
    result = np.vstack((result,output_Three))
    
    return result

# DATA PROCESS AND ENGINEERING

#### TRANSFORMATION AND EXPORTATION

In [ ]:
sample_rate = 22050

In [ ]:
x_Train = []
y_Train = []


for path,category_wav in zip(Main_Train_Data.WAV,Main_Train_Data.CATEGORY):
    
    wav_features = export_function(path)
    
    for indexing in wav_features:
        x_Train.append(indexing)
        y_Train.append(category_wav)

In [ ]:
New_Features_Wav = pd.DataFrame(x_Train)
New_Features_Wav["CATEGORY"] = y_Train

New_Features_Wav.to_csv("New_Wav_Features_Data.csv",index=False)

In [ ]:
New_Features_Wav.head(-1)

In [ ]:
print(New_Features_Wav["CATEGORY"].value_counts())

#### SPLITTING

In [ ]:
OHE_Function = OneHotEncoder()

In [ ]:
Scaler_Function = StandardScaler()

In [ ]:
Part_X = New_Features_Wav.iloc[:,:-1].values
Part_Y = New_Features_Wav["CATEGORY"].values

In [ ]:
print(Part_X.shape)
print(Part_Y.shape)

In [ ]:
Part_Y_Encode = OHE_Function.fit_transform(np.array(Part_Y).reshape(-1,1)).toarray()

In [ ]:
print(Part_Y_Encode.shape)

In [ ]:
xTrain,xTest,yTrain,yTest = train_test_split(Part_X,Part_Y_Encode,train_size=0.9,random_state=42,shuffle=True)

In [ ]:
print(xTrain.shape)
print(yTrain.shape)
print(xTest.shape)
print(yTest.shape)

In [ ]:
xTrain = Scaler_Function.fit_transform(xTrain)
xTest = Scaler_Function.transform(xTest)

In [ ]:
print(xTrain.shape)
print(xTest.shape)

In [ ]:
xTrain = np.expand_dims(xTrain,axis=2)
xTest = np.expand_dims(xTest,axis=2)

In [ ]:
print(xTrain.shape)
print(xTest.shape)

# MODEL STRUCTURE WITH CONV1D

#### PARAMETERS

In [ ]:
output_labels = 10
compile_metrics = ["accuracy"]
compile_loss = "categorical_crossentropy"
compile_optimizer = "adam"

In [ ]:
Call_Back_Early_Stop = tf.keras.callbacks.EarlyStopping(monitor="loss",patience=3,mode="min")
Call_Back_Check = tf.keras.callbacks.ModelCheckpoint(monitor="val_accuracy",
                                                     save_best_only=True,
                                                     filepath="./my_model")

#### STRUCTURE

In [ ]:
Model_Conv1D = Sequential()
#
Model_Conv1D.add(Conv1D(256,5,strides=1,padding="same",activation="relu",input_shape=(xTrain.shape[1],1)))
Model_Conv1D.add(BatchNormalization())
Model_Conv1D.add(MaxPooling1D(3,strides=2,padding="same"))
#
Model_Conv1D.add(Conv1D(256,5,strides=1,padding="same",activation="relu"))
Model_Conv1D.add(Dropout(0.3))
Model_Conv1D.add(MaxPooling1D(3,strides=2,padding="same"))


Model_Conv1D.add(Conv1D(128,5,strides=1,padding="same",activation="relu"))
Model_Conv1D.add(Dropout(0.3))
Model_Conv1D.add(MaxPooling1D(3,strides=2,padding="same"))


Model_Conv1D.add(Conv1D(64,5,strides=1,padding="same",activation="relu"))
Model_Conv1D.add(Dropout(0.3))
Model_Conv1D.add(MaxPooling1D(3,strides=2,padding="same"))
#
Model_Conv1D.add(Flatten())
Model_Conv1D.add(Dense(units=1024, activation='relu'))
Model_Conv1D.add(Dropout(0.3))
#
Model_Conv1D.add(Dense(units=output_labels, activation='softmax'))

In [ ]:
print(Model_Conv1D.summary())

In [ ]:
Model_Conv1D.compile(optimizer=compile_optimizer,loss=compile_loss,metrics=compile_metrics)

In [ ]:
Conv1D_Model = Model_Conv1D.fit(xTrain, yTrain, batch_size=64, epochs=50,
                                validation_data=(xTest, yTest), callbacks=[Call_Back_Early_Stop,Call_Back_Check])

#### PROCESS CHECKING

In [ ]:
plt.style.use("dark_background")

In [ ]:
Grap_Data = pd.DataFrame(Conv1D_Model.history)
Grap_Data.plot()

In [ ]:
plt.plot(Conv1D_Model.history["accuracy"])
plt.plot(Conv1D_Model.history["val_accuracy"])
plt.ylabel("ACCURACY")
plt.legend()
plt.show()

In [ ]:
plt.plot(Conv1D_Model.history["loss"])
plt.plot(Conv1D_Model.history["val_loss"])
plt.ylabel("LOSS")
plt.legend()
plt.show()

#### PREDICTION

In [ ]:
Model_Results = Model_Conv1D.evaluate(xTest,yTest)
print("LOSS:  " + "%.4f" % Model_Results[0])
print("ACCURACY:  " + "%.4f" % Model_Results[1])

In [ ]:
prediction_test = Model_Conv1D.predict(xTest)
y_prediction = OHE_Function.inverse_transform(prediction_test)

yTest = OHE_Function.inverse_transform(yTest)

In [ ]:
print(prediction_test[0:5])
print("---"*20)
print(y_prediction[0:5])
print("---"*20)
print(yTest[0:5])

#### PREDICTION CHECKING

In [ ]:
conf_matrix = confusion_matrix(yTest, y_prediction)
acc_score = accuracy_score(yTest, y_prediction)
class_report = classification_report(yTest, y_prediction)

In [ ]:
sns.heatmap(conf_matrix, linecolor='white', cmap='Blues', linewidth=1, annot=True, fmt='')

plt.title('Confusion Matrix', size=20)
plt.xlabel('Prediction', size=7)
plt.ylabel('Main Labels', size=7)
plt.show()

In [ ]:
print(acc_score)
print("---"*20)
print(class_report)

# MODEL STRUCTURE WITH CONV2D

#### DATA PROCESS AND SPLITTING FOR CONV2D

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(Part_X, Part_Y_Encode, train_size=0.9, random_state = 42, shuffle=True)

In [ ]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 18, 9, 1)
X_test = X_test.reshape(X_test.shape[0], 18, 9, 1)

In [ ]:
print(X_train.shape)
print(X_test.shape)

#### MODEL STRUCTURE

In [ ]:
Call_Back_Check_Conv2D = tf.keras.callbacks.ModelCheckpoint(monitor="val_accuracy",
                                                     save_best_only=True,
                                                     filepath="./my_Conv2D_model")

In [ ]:
Model_Conv2D = Sequential()
#
Model_Conv2D.add(Conv2D(64,(3, 3),padding="same",activation="relu",input_shape=(X_train.shape[1],X_train.shape[2],X_train.shape[3])))
Model_Conv2D.add(MaxPool2D(pool_size=(2, 2)))
#
Model_Conv2D.add(Conv2D(128, (3, 3), padding = "same", activation = "relu"))
Model_Conv2D.add(Dropout(0.3))
Model_Conv2D.add(MaxPool2D(pool_size=(2, 2)))
#
Model_Conv2D.add(Conv2D(256, (3, 3), padding = "same", activation = "relu"))
Model_Conv2D.add(Dropout(0.3))
Model_Conv2D.add(MaxPool2D(pool_size=(2, 2)))
#
Model_Conv2D.add(Flatten())
Model_Conv2D.add(Dense(1024, activation = "relu"))
Model_Conv2D.add(Dropout(0.5))
Model_Conv2D.add(Dense(output_labels, activation = "softmax"))

In [ ]:
Model_Conv2D.compile(optimizer=compile_optimizer,loss=compile_loss,metrics=compile_metrics)

In [ ]:
Conv2D_Model = Model_Conv2D.fit(X_train, Y_train, batch_size=64, epochs=50,
                                validation_data=(X_test, Y_test), callbacks=[Call_Back_Early_Stop,Call_Back_Check_Conv2D])

#### PROCESS CHECKING

In [ ]:
Grap_Data_Conv2D = pd.DataFrame(Conv2D_Model.history)
Grap_Data_Conv2D.plot()

In [ ]:
plt.plot(Conv2D_Model.history["accuracy"])
plt.plot(Conv2D_Model.history["val_accuracy"])
plt.ylabel("ACCURACY")
plt.legend()
plt.show()

In [ ]:
plt.plot(Conv2D_Model.history["loss"])
plt.plot(Conv2D_Model.history["val_loss"])
plt.ylabel("LOSS")
plt.legend()
plt.show()

#### PREDICTION

In [ ]:
Model_Results_Conv2D = Model_Conv2D.evaluate(X_test,Y_test)
print("LOSS:  " + "%.4f" % Model_Results_Conv2D[0])
print("ACCURACY:  " + "%.4f" % Model_Results_Conv2D[1])

In [ ]:
prediction_test_Conv2D = Model_Conv2D.predict(X_test)
prediction_test_Conv2D_Arg = np.argmax(prediction_test_Conv2D,axis=1)

#### PREDICTION CHECKING

In [ ]:
print(prediction_test_Conv2D_Arg)

In [ ]:
y_prediction_Conv2D = OHE_Function.inverse_transform(prediction_test_Conv2D)

In [ ]:
print(y_prediction_Conv2D)

#### END OF THE PROJECT
Please check for the first step: https://www.kaggle.com/brsdincer/urban-sound-prediction-i-ai-ml-process